In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_predict
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import time
import shap

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingRegressor
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

C:\Users\user1\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gwangju_preprocessed = pd.read_csv("Gwangju_preprocess.csv")
busan_preprocessed = pd.read_csv("Busan_preprocess.csv")
daegu_preprocessed = pd.read_csv("Daegu_preprocess.csv")
daejeon_preprocessed = pd.read_csv("Daejeon_preprocess.csv")
incheon_preprocessed = pd.read_csv("Incheon_preprocess.csv")
seoul_preprocessed = pd.read_csv("Seoul_preprocess.csv")

In [3]:
gwangju_preprocessed

,Temp,Humi,WS,WD,Solar,Day_sin,Day_cos,T8,T9,T10,...,T13,T14,T15,T16,T17,T18,D1_Temp,D1_Humi,D1_WS,D1_Solar
0,-7.8,83.0,0.0,0.0,0.00,0.017213,0.999852,1,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.00
1,-5.7,74.0,0.0,0.0,0.22,0.017213,0.999852,0,1,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.00
2,-3.2,61.0,3.5,50.0,1.13,0.017213,0.999852,0,0,1,...,0,0,0,0,0,0,0.0,0.0,0.0,0.00
3,-2.7,57.0,1.2,360.0,1.70,0.017213,0.999852,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.00
4,-1.4,55.0,1.6,50.0,2.00,0.017213,0.999852,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40178,-2.5,76.0,1.4,230.0,1.36,0.017213,0.999852,0,0,0,...,0,1,0,0,0,0,-5.0,85.0,1.5,0.34
40179,-1.9,78.0,1.4,230.0,1.29,0.017213,0.999852,0,0,0,...,0,0,1,0,0,0,-5.0,86.0,1.4,0.21
40180,-2.3,86.0,0.5,200.0,0.70,0.017213,0.999852,0,0,0,...,0,0,0,1,0,0,-5.3,83.0,2.3,0.18
40181,-2.4,94.0,1.7,230.0,0.26,0.017213,0.999852,0,0,0,...,0,0,0,0,1,0,-5.8,91.0,1.8,0.11


1.데이터 분할

In [3]:
gwangju_train = gwangju_preprocessed.iloc[:24112, :]
gwangju_val = gwangju_preprocessed.iloc[24112:32142, :]
gwangju_test = gwangju_preprocessed.iloc[32142:, :]
gwangju_train_val = gwangju_preprocessed.iloc[:32142, :]

busan_train = busan_preprocessed.iloc[:24112, :]
busan_val = busan_preprocessed.iloc[24112:32142, :]
busan_test = busan_preprocessed.iloc[32142:, :]
busan_train_val = busan_preprocessed.iloc[:32142, :]

daegu_train = daegu_preprocessed.iloc[:24112, :]
daegu_val = daegu_preprocessed.iloc[24112:32142, :]
daegu_test = daegu_preprocessed.iloc[32142:, :]
daegu_train_val = daegu_preprocessed.iloc[:32142, :]

daejeon_train = daejeon_preprocessed.iloc[:24112, :]
daejeon_val = daejeon_preprocessed.iloc[24112:32142, :]
daejeon_test = daejeon_preprocessed.iloc[32142:, :]
daejeon_train_val = daejeon_preprocessed.iloc[:32142, :]

incheon_train = incheon_preprocessed.iloc[:24112, :]
incheon_val = incheon_preprocessed.iloc[24112:32142, :]
incheon_test = incheon_preprocessed.iloc[32142:, :]
incheon_train_val = incheon_preprocessed.iloc[:32142, :]

seoul_train = seoul_preprocessed.iloc[:24112, :]
seoul_val = seoul_preprocessed.iloc[24112:32142, :]
seoul_test = seoul_preprocessed.iloc[32142:, :]
seoul_train_val = seoul_preprocessed.iloc[:32142, :]

2.모델 입력에 맞게 데이터 변환   (코드 실행시 각 지역을 한작업으로 수행합니다. 총 6작업으로 나누어 진행하였습니다.)

지역 선택

In [5]:
# Gwangju
X_train_rf_gbm = gwangju_train.drop(columns=['Solar'])
y_train_rf_gbm = gwangju_train['Solar']
X_val_rf_gbm = gwangju_val.drop(columns=['Solar'])
y_val_rf_gbm = gwangju_val['Solar']
X_test_rf_gbm = gwangju_test.drop(columns=['Solar'])
y_test_rf_gbm = gwangju_test['Solar']
X_train_val_rf_gbm = gwangju_train_val.drop(columns=['Solar'])
y_train_val_rf_gbm = gwangju_train_val['Solar']

In [28]:
# Busan
X_train_rf_gbm = busan_train.drop(columns=['Solar'])
y_train_rf_gbm = busan_train['Solar']
X_val_rf_gbm = busan_val.drop(columns=['Solar'])
y_val_rf_gbm = busan_val['Solar']
X_test_rf_gbm = busan_test.drop(columns=['Solar'])
y_test_rf_gbm = busan_test['Solar']
X_train_val_rf_gbm = busan_train_val.drop(columns=['Solar'])
y_train_val_rf_gbm = busan_train_val['Solar']

In [4]:
# Daegu
X_train_rf_gbm = daegu_train.drop(columns=['Solar'])
y_train_rf_gbm = daegu_train['Solar']
X_val_rf_gbm = daegu_val.drop(columns=['Solar'])
y_val_rf_gbm = daegu_val['Solar']
X_test_rf_gbm = daegu_test.drop(columns=['Solar'])
y_test_rf_gbm = daegu_test['Solar']
X_train_val_rf_gbm = daegu_train_val.drop(columns=['Solar'])
y_train_val_rf_gbm = daegu_train_val['Solar']

In [4]:
# Daejeon
X_train_rf_gbm = daejeon_train.drop(columns=['Solar'])
y_train_rf_gbm = daejeon_train['Solar']
X_val_rf_gbm = daejeon_val.drop(columns=['Solar'])
y_val_rf_gbm = daejeon_val['Solar']
X_test_rf_gbm = daejeon_test.drop(columns=['Solar'])
y_test_rf_gbm = daejeon_test['Solar']
X_train_val_rf_gbm = daejeon_train_val.drop(columns=['Solar'])
y_train_val_rf_gbm = daejeon_train_val['Solar']

In [4]:
# Incheon
X_train_rf_gbm = incheon_train.drop(columns=['Solar'])
y_train_rf_gbm = incheon_train['Solar']
X_val_rf_gbm = incheon_val.drop(columns=['Solar'])
y_val_rf_gbm = incheon_val['Solar']
X_test_rf_gbm = incheon_test.drop(columns=['Solar'])
y_test_rf_gbm = incheon_test['Solar']
X_train_val_rf_gbm = incheon_train_val.drop(columns=['Solar'])
y_train_val_rf_gbm = incheon_train_val['Solar']

In [4]:
# Seoul
X_train_rf_gbm = seoul_train.drop(columns=['Solar'])
y_train_rf_gbm = seoul_train['Solar']
X_val_rf_gbm = seoul_val.drop(columns=['Solar'])
y_val_rf_gbm = seoul_val['Solar']
X_test_rf_gbm = seoul_test.drop(columns=['Solar'])
y_test_rf_gbm = seoul_test['Solar']
X_train_val_rf_gbm = seoul_train_val.drop(columns=['Solar'])
y_train_val_rf_gbm = seoul_train_val['Solar']

모든 지역 공통

In [5]:
# XGBoost를 위한 데이터 준비 공통
import xgboost as xgb

# 범주형 변수 설정
dtrain = xgb.DMatrix(X_train_rf_gbm, label=y_train_rf_gbm)
dval = xgb.DMatrix(X_val_rf_gbm, label=y_val_rf_gbm)
dtest = xgb.DMatrix(X_test_rf_gbm, label=y_test_rf_gbm)
dtrain_val = xgb.DMatrix(X_train_val_rf_gbm, label=y_train_val_rf_gbm)

# LightGBM을 위한 데이터 준비 공통
import lightgbm as lgb

ltrain = lgb.Dataset(X_train_rf_gbm, label=y_train_rf_gbm)
lval = lgb.Dataset(X_val_rf_gbm, label=y_val_rf_gbm)
ltest = lgb.Dataset(X_test_rf_gbm, label=y_test_rf_gbm)
ltrain_val = lgb.Dataset(X_train_val_rf_gbm, label=y_train_val_rf_gbm)

# CatBoost를 위한 데이터 준비 공통
import catboost as cb

ctrain = cb.Pool(data=X_train_rf_gbm, label=y_train_rf_gbm)
cval = cb.Pool(data=X_val_rf_gbm, label=y_val_rf_gbm)
ctest = cb.Pool(data=X_test_rf_gbm, label=y_test_rf_gbm)
ctrain_val = cb.Pool(data=X_train_val_rf_gbm, label=y_train_val_rf_gbm)

In [10]:
pip install optuna

     -------------------------------------- 364.4/364.4 KB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 3.8 MB/s eta 0:00:00
     -------------------------------------- 233.5/233.5 KB 2.4 MB/s eta 0:00:00
     -------------------------------------- 161.8/161.8 KB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 78.6/78.6 KB 2.2 MB/s eta 0:00:00
     -------------------------------------- 298.4/298.4 KB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\양태영\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [6]:
# optuna + seed 고정 / train으로 학습 val을 통해 평가

import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Set random seed for reproducibility
SEED = 42  # 랜덤 시드 설정 추가

# Random Forest Objective
def objective_rf(trial):
    
    np.random.seed(SEED)  # 랜덤 시드 설정 추가

    
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [128]),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    }
    model = RandomForestRegressor(**params, random_state=SEED)
    model.fit(X_train_rf_gbm, y_train_rf_gbm)
    predictions = model.predict(X_val_rf_gbm)
    rmse = np.sqrt(mean_squared_error(y_val_rf_gbm, predictions))
    return rmse

# Gradient Boosting Objective
def objective_gbm(trial):
    
    np.random.seed(SEED)
    
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [100, 500]),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, step=0.01),
        'max_depth': trial.suggest_int('max_depth', 5, 10),
        'loss': trial.suggest_categorical('loss', ['quantile', 'huber'])
    }
    model = GradientBoostingRegressor(**params, random_state=SEED)
    model.fit(X_train_rf_gbm, y_train_rf_gbm)
    predictions = model.predict(X_val_rf_gbm)
    rmse = np.sqrt(mean_squared_error(y_val_rf_gbm, predictions))
    return rmse

# XGBoost Objective for Optuna
def objective_xgb(trial):
    # Hyperparameter suggestion
    np.random.seed(SEED)

    params = {
        'objective': 'reg:squarederror',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, step=0.01),
        'max_depth': trial.suggest_int('max_depth', 6, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step = 0.05),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step = 0.05),
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'random_state': SEED  # XGBoost의 랜덤 시드 설정 추가
    }
    # Separate num_boost_round from params
    num_boost_round = trial.suggest_categorical('n_estimators', [250, 500, 1000])

    # XGBoost Training with DMatrix
    model = xgb.train(params, dtrain, num_boost_round=num_boost_round)

    # Predict on test set
    predictions = model.predict(dval)

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(dval.get_label(), predictions))
    return rmse

# LightGBM Objective
def objective_lgb(trial):
    
    np.random.seed(SEED)
    
    params = {
        'objective': 'regression',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, step=0.01),
        'num_leaves': trial.suggest_int('num_leaves', 64, 64),
        'subsample': trial.suggest_categorical('subsample', [0.5]),
        'feature_fraction': trial.suggest_categorical('feature_fraction', [1.0]),
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
        'seed': SEED  # LightGBM의 랜덤 시드 설정 추가
    }
    num_boost_round = trial.suggest_categorical('n_estimators', [1000, 1500])
    model = lgb.train(params, ltrain, num_boost_round=num_boost_round)
    predictions = model.predict(X_val_rf_gbm)
    rmse = np.sqrt(mean_squared_error(y_val_rf_gbm, predictions))
    return rmse

# CatBoost Objective
def objective_cat(trial):
    
    np.random.seed(SEED)
    
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.1, step = 0.01),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 9),
        'random_state': SEED
    }
    model = cb.CatBoostRegressor(**params,verbose=0)
    model.fit(ctrain)
    predictions = model.predict(cval)
    rmse = np.sqrt(mean_squared_error(y_val_rf_gbm, predictions))
    return rmse

3. Optuna 하이퍼파라미터 최적화(train : val)

In [7]:
study_rf = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study_rf.optimize(objective_rf, n_trials=100)

[I 2024-12-05 19:14:52,178] A new study created in memory with name: no-name-15d08627-4007-4466-9b49-dbd767bd0674
[I 2024-12-05 19:14:58,038] Trial 0 finished with value: 0.3892836685749838 and parameters: {'n_estimators': 128, 'max_features': 'log2'}. Best is trial 0 with value: 0.3892836685749838.
[I 2024-12-05 19:15:04,020] Trial 1 finished with value: 0.3892836685749838 and parameters: {'n_estimators': 128, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.3892836685749838.
[I 2024-12-05 19:15:09,843] Trial 2 finished with value: 0.3892836685749838 and parameters: {'n_estimators': 128, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.3892836685749838.
[I 2024-12-05 19:15:15,830] Trial 3 finished with value: 0.3892836685749838 and parameters: {'n_estimators': 128, 'max_features': 'log2'}. Best is trial 0 with value: 0.3892836685749838.
[I 2024-12-05 19:15:22,313] Trial 4 finished with value: 0.3892836685749838 and parameters: {'n_estimators': 128, 'max_features': 'log2'}.

In [8]:
study_gbm = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study_gbm.optimize(objective_gbm, n_trials=100)

[I 2024-12-05 19:24:30,759] A new study created in memory with name: no-name-762ed100-5321-4147-aa9d-fad436e1fad3
[I 2024-12-05 19:25:16,296] Trial 0 finished with value: 0.46962303995892607 and parameters: {'n_estimators': 500, 'learning_rate': 0.08, 'max_depth': 8, 'loss': 'quantile'}. Best is trial 0 with value: 0.46962303995892607.
[I 2024-12-05 19:26:34,176] Trial 1 finished with value: 0.3795400549909196 and parameters: {'n_estimators': 500, 'learning_rate': 0.06999999999999999, 'max_depth': 9, 'loss': 'huber'}. Best is trial 1 with value: 0.3795400549909196.
[I 2024-12-05 19:26:42,301] Trial 2 finished with value: 0.4402242209687781 and parameters: {'n_estimators': 100, 'learning_rate': 0.02, 'max_depth': 6, 'loss': 'huber'}. Best is trial 1 with value: 0.3795400549909196.
[I 2024-12-05 19:26:48,559] Trial 3 finished with value: 0.3778112418110285 and parameters: {'n_estimators': 100, 'learning_rate': 0.06999999999999999, 'max_depth': 5, 'loss': 'huber'}. Best is trial 3 with va

In [9]:
study_xgb = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study_xgb.optimize(objective_xgb, n_trials=100)

[I 2024-12-05 20:26:34,432] A new study created in memory with name: no-name-24b7fd49-557a-4064-aae5-3adac452aee1
[I 2024-12-05 20:26:38,445] Trial 0 finished with value: 0.37635311484336853 and parameters: {'learning_rate': 0.04, 'max_depth': 10, 'subsample': 0.9, 'colsample_bytree': 0.8, 'booster': 'gbtree', 'n_estimators': 500}. Best is trial 0 with value: 0.37635311484336853.
[I 2024-12-05 20:26:40,546] Trial 1 finished with value: 0.3808160126209259 and parameters: {'learning_rate': 0.08, 'max_depth': 6, 'subsample': 1.0, 'colsample_bytree': 0.95, 'booster': 'gbtree', 'n_estimators': 1000}. Best is trial 0 with value: 0.37635311484336853.
[I 2024-12-05 20:27:39,455] Trial 2 finished with value: 0.3749759793281555 and parameters: {'learning_rate': 0.05, 'max_depth': 7, 'subsample': 0.8, 'colsample_bytree': 0.55, 'booster': 'dart', 'n_estimators': 500}. Best is trial 2 with value: 0.3749759793281555.
[I 2024-12-05 20:27:41,427] Trial 3 finished with value: 0.3812463581562042 and par

In [10]:
study_lgb = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study_lgb.optimize(objective_lgb, n_trials=100)

[I 2024-12-05 21:17:47,573] A new study created in memory with name: no-name-1138157f-a12b-47ab-90ff-c1ab44e8328f


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002222 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1840
[LightGBM] [Info] Number of data points in the train set: 24112, number of used features: 21
[LightGBM] [Info] Start training from score 1.093214


[I 2024-12-05 21:17:50,634] Trial 0 finished with value: 0.37408119323378836 and parameters: {'learning_rate': 0.04, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1000}. Best is trial 0 with value: 0.37408119323378836.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000265 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1840
[LightGBM] [Info] Number of data points in the train set: 24112, number of used features: 21
[LightGBM] [Info] Start training from score 1.093214


[I 2024-12-05 21:18:15,446] Trial 1 finished with value: 0.39065045840043805 and parameters: {'learning_rate': 0.02, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'dart', 'n_estimators': 1500}. Best is trial 0 with value: 0.37408119323378836.


...

[I 2024-12-05 21:25:48,185] Trial 97 finished with value: 0.366419897466168 and parameters: {'learning_rate': 0.01, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1000}. Best is trial 2 with value: 0.366419897466168.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000957 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1840
[LightGBM] [Info] Number of data points in the train set: 24112, number of used features: 21
[LightGBM] [Info] Start training from score 1.093214


[I 2024-12-05 21:25:51,044] Trial 98 finished with value: 0.366419897466168 and parameters: {'learning_rate': 0.01, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1000}. Best is trial 2 with value: 0.366419897466168.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001019 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1840
[LightGBM] [Info] Number of data points in the train set: 24112, number of used features: 21
[LightGBM] [Info] Start training from score 1.093214


[I 2024-12-05 21:25:53,557] Trial 99 finished with value: 0.36796978697235616 and parameters: {'learning_rate': 0.02, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1000}. Best is trial 2 with value: 0.366419897466168.


In [11]:
study_cat = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study_cat.optimize(objective_cat, n_trials=100)

[I 2024-12-05 21:25:53,565] A new study created in memory with name: no-name-d1a79d8d-054f-4277-99cc-927eeefa7411
[I 2024-12-05 21:26:13,271] Trial 0 finished with value: 0.3700971815157761 and parameters: {'learning_rate': 0.05, 'depth': 10, 'l2_leaf_reg': 7}. Best is trial 0 with value: 0.3700971815157761.
[I 2024-12-05 21:26:17,921] Trial 1 finished with value: 0.3704788625340517 and parameters: {'learning_rate': 0.07, 'depth': 5, 'l2_leaf_reg': 2}. Best is trial 0 with value: 0.3700971815157761.
[I 2024-12-05 21:26:37,197] Trial 2 finished with value: 0.36798241824514444 and parameters: {'learning_rate': 0.03, 'depth': 10, 'l2_leaf_reg': 6}. Best is trial 2 with value: 0.36798241824514444.
[I 2024-12-05 21:26:40,305] Trial 3 finished with value: 0.372522275676749 and parameters: {'learning_rate': 0.08, 'depth': 4, 'l2_leaf_reg': 9}. Best is trial 2 with value: 0.36798241824514444.
[I 2024-12-05 21:26:43,730] Trial 4 finished with value: 0.37202741693052227 and parameters: {'learnin

4. 모델 학습(데이터 train + val : test)

광주

In [12]:

# Random Forest
best_params_rf = study_rf.best_params
print("Best parameters for Random Forest:", best_params_rf)

# 모델 학습 및 저장
start_time = time.time()
final_model_rf = RandomForestRegressor(**best_params_rf, random_state=SEED)
final_model_rf.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_rf = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_rf, "./model_/gwangju_rf.pkl")

# 모델 성능 평가
final_predictions_rf = final_model_rf.predict(X_test_rf_gbm)
final_rmse_rf = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_rf))
final_mse_rf = mean_squared_error(y_test_rf_gbm, final_predictions_rf)
final_mae_rf = mean_absolute_error(y_test_rf_gbm, final_predictions_rf)
final_r2_rf = r2_score(y_test_rf_gbm, final_predictions_rf)

print(f"Random Forest - Training Time: {training_time_rf:.2f}s, RMSE: {final_rmse_rf:.4f}, MSE: {final_mse_rf:.4f}, MAE: {final_mae_rf:.4f}, R2: {final_r2_rf:.4f}")

Best parameters for Random Forest: {'n_estimators': 128, 'max_features': 'log2'}
Random Forest - Training Time: 7.67s, RMSE: 0.4227, MSE: 0.1787, MAE: 0.3185, R2: 0.8100


In [20]:
# Gradient Boosting
best_params_gbm = study_gbm.best_params
print("Best parameters for Gradient Boosting:", best_params_gbm)

start_time = time.time()
final_model_gbm = GradientBoostingRegressor(**best_params_gbm)
final_model_gbm.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_gbm = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_gbm, "./model_/gwangju_gbm.pkl")

final_predictions_gbm = final_model_gbm.predict(X_test_rf_gbm)
final_rmse_gbm = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_gbm))
final_mse_gbm = mean_squared_error(y_test_rf_gbm, final_predictions_gbm)
final_mae_gbm = mean_absolute_error(y_test_rf_gbm, final_predictions_gbm)
final_r2_gbm = r2_score(y_test_rf_gbm, final_predictions_gbm)
print(f"Gradient Boosting - Training Time: {training_time_gbm:.2f}s, RMSE: {final_rmse_gbm:.4f}, MSE: {final_mse_gbm:.4f}, MAE: {final_mae_gbm:.4f}, R2: {final_r2_gbm:.4f}")

Best parameters for Gradient Boosting: {'n_estimators': 500, 'learning_rate': 0.060000000000000005, 'max_depth': 5, 'loss': 'huber'}
Gradient Boosting - Training Time: 41.51s, RMSE: 0.4007, MSE: 0.1605, MAE: 0.2944, R2: 0.8293


In [22]:
# XGBoost
best_params_xgb = study_xgb.best_params
print("Best parameters for XGBoost:", best_params_xgb)

best_num_boost_round = best_params_xgb.pop('n_estimators')
start_time = time.time()
final_model_xgb = xgb.train(best_params_xgb, dtrain_val, num_boost_round=best_num_boost_round)
training_time_xgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_xgb, "./model_/gwangju_xgb.pkl")

# 예측 (test 데이터로 예측)
final_predictions_xgb = final_model_xgb.predict(dtest)

# 평가 지표 계산
final_rmse_xgb = np.sqrt(mean_squared_error(dtest.get_label(), final_predictions_xgb))
final_mse_xgb = mean_squared_error(dtest.get_label(), final_predictions_xgb)
final_mae_xgb = mean_absolute_error(dtest.get_label(), final_predictions_xgb)
final_r2_xgb = r2_score(dtest.get_label(), final_predictions_xgb)

# 결과 출력
print(f"XGBoost - Training Time: {training_time_xgb:.2f}s, RMSE: {final_rmse_xgb:.4f}, MSE: {final_mse_xgb:.4f}, MAE: {final_mae_xgb:.4f}, R2: {final_r2_xgb:.4f}")

Best parameters for XGBoost: {'learning_rate': 0.04, 'max_depth': 8, 'subsample': 0.65, 'colsample_bytree': 0.7, 'booster': 'gbtree', 'n_estimators': 250}
XGBoost - Training Time: 1.58s, RMSE: 0.4064, MSE: 0.1652, MAE: 0.3018, R2: 0.8243


In [18]:
# LightGBM
best_params_lgb = study_lgb.best_params
print("Best parameters for LightGBM:", best_params_lgb)

start_time = time.time()
final_model_lgb = lgb.train(best_params_lgb, ltrain_val, num_boost_round=best_params_lgb['n_estimators'])
training_time_lgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_lgb, "./model_/gwangju_lgb.pkl")

# 예측 (test 데이터로 예측)
final_predictions_lgb = final_model_lgb.predict(X_test_rf_gbm)  # X_test_rf_gbm 사용

# 평가 지표 계산
final_rmse_lgb = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_lgb))
final_mse_lgb = mean_squared_error(y_test_rf_gbm, final_predictions_lgb)
final_mae_lgb = mean_absolute_error(y_test_rf_gbm, final_predictions_lgb)
final_r2_lgb = r2_score(y_test_rf_gbm, final_predictions_lgb)

# 결과 출력
print(f"LightGBM - Training Time: {training_time_lgb:.2f}s, RMSE: {final_rmse_lgb:.4f}, MSE: {final_mse_lgb:.4f}, MAE: {final_mae_lgb:.4f}, R2: {final_r2_lgb:.4f}")

C:\Users\user1\AppData\Roaming\Python\Python39\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Best parameters for LightGBM: {'learning_rate': 0.02, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1000}
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000564 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1828
[LightGBM] [Info] Number of data points in the train set: 32142, number of used features: 21
[LightGBM] [Info] Start training from score 1.267165
LightGBM - Training Time: 2.98s, RMSE: 0.4012, MSE: 0.1609, MAE: 0.2960, R2: 0.8289


In [24]:
# CatBoost
best_params_cat = study_cat.best_params
print("Best parameters for CatBoost:", best_params_cat)

start_time = time.time()
final_model_cat = cb.CatBoostRegressor(**best_params_cat, verbose=0, random_seed=SEED)
final_model_cat.fit(ctrain_val)
training_time_cat = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_cat, "./model_/gwangju_cat.pkl")

final_predictions_cat = final_model_cat.predict(ctest)
final_rmse_cat = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_cat))
final_mse_cat = mean_squared_error(y_test_rf_gbm, final_predictions_cat)
final_mae_cat = mean_absolute_error(y_test_rf_gbm, final_predictions_cat)
final_r2_cat = r2_score(y_test_rf_gbm, final_predictions_cat)
print(f"CatBoost - Training Time: {training_time_cat:.2f}s, RMSE: {final_rmse_cat:.4f}, MSE: {final_mse_cat:.4f}, MAE: {final_mae_cat:.4f}, R2: {final_r2_cat:.4f}")

Best parameters for CatBoost: {'learning_rate': 0.04, 'depth': 9, 'l2_leaf_reg': 6}
CatBoost - Training Time: 9.90s, RMSE: 0.3998, MSE: 0.1599, MAE: 0.2971, R2: 0.8300


부산


In [38]:
# Random Forest
best_params_rf = study_rf.best_params
print("Best parameters for Random Forest:", best_params_rf)

# 모델 학습 및 저장
start_time = time.time()
final_model_rf = RandomForestRegressor(**best_params_rf, random_state=SEED)
final_model_rf.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_rf = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_rf, "./model_/busan_rf.pkl")

# 모델 성능 평가
final_predictions_rf = final_model_rf.predict(X_test_rf_gbm)
final_rmse_rf = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_rf))
final_mse_rf = mean_squared_error(y_test_rf_gbm, final_predictions_rf)
final_mae_rf = mean_absolute_error(y_test_rf_gbm, final_predictions_rf)
final_r2_rf = r2_score(y_test_rf_gbm, final_predictions_rf)

print(f"Random Forest - Training Time: {training_time_rf:.2f}s, RMSE: {final_rmse_rf:.4f}, MSE: {final_mse_rf:.4f}, MAE: {final_mae_rf:.4f}, R2: {final_r2_rf:.4f}")

Best parameters for Random Forest: {'n_estimators': 128, 'max_features': 'log2'}
Random Forest - Training Time: 7.96s, RMSE: 0.4313, MSE: 0.1861, MAE: 0.3223, R2: 0.8177


In [39]:
# Gradient Boosting
best_params_gbm = study_gbm.best_params
print("Best parameters for Gradient Boosting:", best_params_gbm)

start_time = time.time()
final_model_gbm = GradientBoostingRegressor(**best_params_gbm)
final_model_gbm.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_gbm = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_gbm, "./model_/busan_gbm.pkl")

final_predictions_gbm = final_model_gbm.predict(X_test_rf_gbm)
final_rmse_gbm = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_gbm))
final_mse_gbm = mean_squared_error(y_test_rf_gbm, final_predictions_gbm)
final_mae_gbm = mean_absolute_error(y_test_rf_gbm, final_predictions_gbm)
final_r2_gbm = r2_score(y_test_rf_gbm, final_predictions_gbm)
print(f"Gradient Boosting - Training Time: {training_time_gbm:.2f}s, RMSE: {final_rmse_gbm:.4f}, MSE: {final_mse_gbm:.4f}, MAE: {final_mae_gbm:.4f}, R2: {final_r2_gbm:.4f}")

Best parameters for Gradient Boosting: {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 8, 'loss': 'huber'}
Gradient Boosting - Training Time: 17.43s, RMSE: 0.4177, MSE: 0.1744, MAE: 0.2997, R2: 0.8291


In [40]:
# XGBoost
best_params_xgb = study_xgb.best_params
print("Best parameters for XGBoost:", best_params_xgb)

best_num_boost_round = best_params_xgb.pop('n_estimators')
start_time = time.time()
final_model_xgb = xgb.train(best_params_xgb, dtrain_val, num_boost_round=best_num_boost_round)
training_time_xgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_xgb, "./model_/busan_xgb.pkl")

# 예측 (test 데이터로 예측)
final_predictions_xgb = final_model_xgb.predict(dtest)

# 평가 지표 계산
final_rmse_xgb = np.sqrt(mean_squared_error(dtest.get_label(), final_predictions_xgb))
final_mse_xgb = mean_squared_error(dtest.get_label(), final_predictions_xgb)
final_mae_xgb = mean_absolute_error(dtest.get_label(), final_predictions_xgb)
final_r2_xgb = r2_score(dtest.get_label(), final_predictions_xgb)

# 결과 출력
print(f"XGBoost - Training Time: {training_time_xgb:.2f}s, RMSE: {final_rmse_xgb:.4f}, MSE: {final_mse_xgb:.4f}, MAE: {final_mae_xgb:.4f}, R2: {final_r2_xgb:.4f}")

Best parameters for XGBoost: {'learning_rate': 0.03, 'max_depth': 9, 'subsample': 0.9, 'colsample_bytree': 0.9, 'booster': 'gbtree', 'n_estimators': 250}
XGBoost - Training Time: 1.95s, RMSE: 0.4063, MSE: 0.1651, MAE: 0.2980, R2: 0.8383


In [34]:
# LightGBM
best_params_lgb = study_lgb.best_params
print("Best parameters for LightGBM:", best_params_lgb)

start_time = time.time()
final_model_lgb = lgb.train(best_params_lgb, ltrain_val, num_boost_round=best_params_lgb['n_estimators'])
training_time_lgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_lgb, "./model_/busan_lgb.pkl")

# 예측 (test 데이터로 예측)
final_predictions_lgb = final_model_lgb.predict(X_test_rf_gbm)  # X_test_rf_gbm 사용

# 평가 지표 계산
final_rmse_lgb = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_lgb))
final_mse_lgb = mean_squared_error(y_test_rf_gbm, final_predictions_lgb)
final_mae_lgb = mean_absolute_error(y_test_rf_gbm, final_predictions_lgb)
final_r2_lgb = r2_score(y_test_rf_gbm, final_predictions_lgb)

# 결과 출력
print(f"LightGBM - Training Time: {training_time_lgb:.2f}s, RMSE: {final_rmse_lgb:.4f}, MSE: {final_mse_lgb:.4f}, MAE: {final_mae_lgb:.4f}, R2: {final_r2_lgb:.4f}")

C:\Users\user1\AppData\Roaming\Python\Python39\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Best parameters for LightGBM: {'learning_rate': 0.01, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1000}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001679 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2253
[LightGBM] [Info] Number of data points in the train set: 32142, number of used features: 21
[LightGBM] [Info] Start training from score 1.253945
LightGBM - Training Time: 2.74s, RMSE: 0.4072, MSE: 0.1658, MAE: 0.2976, R2: 0.8375


In [32]:
# CatBoost
best_params_cat = study_cat.best_params
print("Best parameters for CatBoost:", best_params_cat)

start_time = time.time()
final_model_cat = cb.CatBoostRegressor(**best_params_cat, verbose=0, random_seed=SEED)
final_model_cat.fit(ctrain_val)
training_time_cat = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_cat, "./model_/busan_cat.pkl")

final_predictions_cat = final_model_cat.predict(ctest)
final_rmse_cat = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_cat))
final_mse_cat = mean_squared_error(y_test_rf_gbm, final_predictions_cat)
final_mae_cat = mean_absolute_error(y_test_rf_gbm, final_predictions_cat)
final_r2_cat = r2_score(y_test_rf_gbm, final_predictions_cat)
print(f"CatBoost - Training Time: {training_time_cat:.2f}s, RMSE: {final_rmse_cat:.4f}, MSE: {final_mse_cat:.4f}, MAE: {final_mae_cat:.4f}, R2: {final_r2_cat:.4f}")

Best parameters for CatBoost: {'learning_rate': 0.03, 'depth': 10, 'l2_leaf_reg': 5}
CatBoost - Training Time: 21.93s, RMSE: 0.4072, MSE: 0.1658, MAE: 0.3008, R2: 0.8376


대구

In [12]:
# Random Forest
best_params_rf = study_rf.best_params
print("Best parameters for Random Forest:", best_params_rf)

# 모델 학습 및 저장
start_time = time.time()
final_model_rf = RandomForestRegressor(**best_params_rf, random_state=SEED)
final_model_rf.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_rf = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_rf, "./model_/daegu_rf.pkl")

# 모델 성능 평가
final_predictions_rf = final_model_rf.predict(X_test_rf_gbm)
final_rmse_rf = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_rf))
final_mse_rf = mean_squared_error(y_test_rf_gbm, final_predictions_rf)
final_mae_rf = mean_absolute_error(y_test_rf_gbm, final_predictions_rf)
final_r2_rf = r2_score(y_test_rf_gbm, final_predictions_rf)

print(f"Random Forest - Training Time: {training_time_rf:.2f}s, RMSE: {final_rmse_rf:.4f}, MSE: {final_mse_rf:.4f}, MAE: {final_mae_rf:.4f}, R2: {final_r2_rf:.4f}")

Best parameters for Random Forest: {'n_estimators': 128, 'max_features': 'log2'}
Random Forest - Training Time: 7.77s, RMSE: 0.3806, MSE: 0.1448, MAE: 0.2757, R2: 0.8341


In [13]:
# Gradient Boosting
best_params_gbm = study_gbm.best_params
print("Best parameters for Gradient Boosting:", best_params_gbm)

start_time = time.time()
final_model_gbm = GradientBoostingRegressor(**best_params_gbm)
final_model_gbm.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_gbm = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_gbm, "./model_/daegu_gbm.pkl")

final_predictions_gbm = final_model_gbm.predict(X_test_rf_gbm)
final_rmse_gbm = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_gbm))
final_mse_gbm = mean_squared_error(y_test_rf_gbm, final_predictions_gbm)
final_mae_gbm = mean_absolute_error(y_test_rf_gbm, final_predictions_gbm)
final_r2_gbm = r2_score(y_test_rf_gbm, final_predictions_gbm)
print(f"Gradient Boosting - Training Time: {training_time_gbm:.2f}s, RMSE: {final_rmse_gbm:.4f}, MSE: {final_mse_gbm:.4f}, MAE: {final_mae_gbm:.4f}, R2: {final_r2_gbm:.4f}")

Best parameters for Gradient Boosting: {'n_estimators': 500, 'learning_rate': 0.04, 'max_depth': 6, 'loss': 'huber'}
Gradient Boosting - Training Time: 51.05s, RMSE: 0.3775, MSE: 0.1425, MAE: 0.2665, R2: 0.8367


In [14]:
# XGBoost
best_params_xgb = study_xgb.best_params
print("Best parameters for XGBoost:", best_params_xgb)

best_num_boost_round = best_params_xgb.pop('n_estimators')
start_time = time.time()
final_model_xgb = xgb.train(best_params_xgb, dtrain_val, num_boost_round=best_num_boost_round)
training_time_xgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_xgb, "./model_/daegu_xgb.pkl")

# 예측 (test 데이터로 예측)
final_predictions_xgb = final_model_xgb.predict(dtest)

# 평가 지표 계산
final_rmse_xgb = np.sqrt(mean_squared_error(dtest.get_label(), final_predictions_xgb))
final_mse_xgb = mean_squared_error(dtest.get_label(), final_predictions_xgb)
final_mae_xgb = mean_absolute_error(dtest.get_label(), final_predictions_xgb)
final_r2_xgb = r2_score(dtest.get_label(), final_predictions_xgb)

# 결과 출력
print(f"XGBoost - Training Time: {training_time_xgb:.2f}s, RMSE: {final_rmse_xgb:.4f}, MSE: {final_mse_xgb:.4f}, MAE: {final_mae_xgb:.4f}, R2: {final_r2_xgb:.4f}")

Best parameters for XGBoost: {'learning_rate': 0.02, 'max_depth': 8, 'subsample': 0.65, 'colsample_bytree': 0.8500000000000001, 'booster': 'gbtree', 'n_estimators': 500}
XGBoost - Training Time: 2.68s, RMSE: 0.3725, MSE: 0.1388, MAE: 0.2650, R2: 0.8410


In [15]:
# LightGBM
best_params_lgb = study_lgb.best_params
print("Best parameters for LightGBM:", best_params_lgb)

start_time = time.time()
final_model_lgb = lgb.train(best_params_lgb, ltrain_val, num_boost_round=best_params_lgb['n_estimators'])
training_time_lgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_lgb, "./model_/daegu_lgb.pkl")

# 예측 (test 데이터로 예측)
final_predictions_lgb = final_model_lgb.predict(X_test_rf_gbm)  # X_test_rf_gbm 사용

# 평가 지표 계산
final_rmse_lgb = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_lgb))
final_mse_lgb = mean_squared_error(y_test_rf_gbm, final_predictions_lgb)
final_mae_lgb = mean_absolute_error(y_test_rf_gbm, final_predictions_lgb)
final_r2_lgb = r2_score(y_test_rf_gbm, final_predictions_lgb)

# 결과 출력
print(f"LightGBM - Training Time: {training_time_lgb:.2f}s, RMSE: {final_rmse_lgb:.4f}, MSE: {final_mse_lgb:.4f}, MAE: {final_mae_lgb:.4f}, R2: {final_r2_lgb:.4f}")

Best parameters for LightGBM: {'learning_rate': 0.060000000000000005, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'dart', 'n_estimators': 1000}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001486 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1890
[LightGBM] [Info] Number of data points in the train set: 32142, number of used features: 21
[LightGBM] [Info] Start training from score 1.313424


C:\Users\user1\AppData\Roaming\Python\Python39\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


LightGBM - Training Time: 16.32s, RMSE: 0.3785, MSE: 0.1433, MAE: 0.2703, R2: 0.8359


In [16]:
# CatBoost
best_params_cat = study_cat.best_params
print("Best parameters for CatBoost:", best_params_cat)

start_time = time.time()
final_model_cat = cb.CatBoostRegressor(**best_params_cat, verbose=0, random_seed=SEED)
final_model_cat.fit(ctrain_val)
training_time_cat = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_cat, "./model_/daegu_cat.pkl")

final_predictions_cat = final_model_cat.predict(ctest)
final_rmse_cat = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_cat))
final_mse_cat = mean_squared_error(y_test_rf_gbm, final_predictions_cat)
final_mae_cat = mean_absolute_error(y_test_rf_gbm, final_predictions_cat)
final_r2_cat = r2_score(y_test_rf_gbm, final_predictions_cat)
print(f"CatBoost - Training Time: {training_time_cat:.2f}s, RMSE: {final_rmse_cat:.4f}, MSE: {final_mse_cat:.4f}, MAE: {final_mae_cat:.4f}, R2: {final_r2_cat:.4f}")

Best parameters for CatBoost: {'learning_rate': 0.03, 'depth': 9, 'l2_leaf_reg': 9}
CatBoost - Training Time: 10.17s, RMSE: 0.3694, MSE: 0.1364, MAE: 0.2632, R2: 0.8437


대전

In [12]:
# Random Forest
best_params_rf = study_rf.best_params
print("Best parameters for Random Forest:", best_params_rf)

# 모델 학습 및 저장
start_time = time.time()
final_model_rf = RandomForestRegressor(**best_params_rf, random_state=SEED)
final_model_rf.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_rf = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_rf, "./model_/daejeon_rf.pkl")

# 모델 성능 평가
final_predictions_rf = final_model_rf.predict(X_test_rf_gbm)
final_rmse_rf = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_rf))
final_mse_rf = mean_squared_error(y_test_rf_gbm, final_predictions_rf)
final_mae_rf = mean_absolute_error(y_test_rf_gbm, final_predictions_rf)
final_r2_rf = r2_score(y_test_rf_gbm, final_predictions_rf)

print(f"Random Forest - Training Time: {training_time_rf:.2f}s, RMSE: {final_rmse_rf:.4f}, MSE: {final_mse_rf:.4f}, MAE: {final_mae_rf:.4f}, R2: {final_r2_rf:.4f}")

Best parameters for Random Forest: {'n_estimators': 128, 'max_features': 'log2'}
Random Forest - Training Time: 7.72s, RMSE: 0.4188, MSE: 0.1754, MAE: 0.3085, R2: 0.8297


In [13]:
# Gradient Boosting
best_params_gbm = study_gbm.best_params
print("Best parameters for Gradient Boosting:", best_params_gbm)

start_time = time.time()
final_model_gbm = GradientBoostingRegressor(**best_params_gbm)
final_model_gbm.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_gbm = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_gbm, "./model_/daejeon_gbm.pkl")

final_predictions_gbm = final_model_gbm.predict(X_test_rf_gbm)
final_rmse_gbm = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_gbm))
final_mse_gbm = mean_squared_error(y_test_rf_gbm, final_predictions_gbm)
final_mae_gbm = mean_absolute_error(y_test_rf_gbm, final_predictions_gbm)
final_r2_gbm = r2_score(y_test_rf_gbm, final_predictions_gbm)
print(f"Gradient Boosting - Training Time: {training_time_gbm:.2f}s, RMSE: {final_rmse_gbm:.4f}, MSE: {final_mse_gbm:.4f}, MAE: {final_mae_gbm:.4f}, R2: {final_r2_gbm:.4f}")

Best parameters for Gradient Boosting: {'n_estimators': 500, 'learning_rate': 0.03, 'max_depth': 7, 'loss': 'huber'}
Gradient Boosting - Training Time: 62.62s, RMSE: 0.4053, MSE: 0.1643, MAE: 0.2904, R2: 0.8404


In [14]:
# XGBoost
best_params_xgb = study_xgb.best_params
print("Best parameters for XGBoost:", best_params_xgb)

best_num_boost_round = best_params_xgb.pop('n_estimators')
start_time = time.time()
final_model_xgb = xgb.train(best_params_xgb, dtrain_val, num_boost_round=best_num_boost_round)
training_time_xgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_xgb, "./model_/daejeon_xgb.pkl")

# 예측 (test 데이터로 예측)
final_predictions_xgb = final_model_xgb.predict(dtest)

# 평가 지표 계산
final_rmse_xgb = np.sqrt(mean_squared_error(dtest.get_label(), final_predictions_xgb))
final_mse_xgb = mean_squared_error(dtest.get_label(), final_predictions_xgb)
final_mae_xgb = mean_absolute_error(dtest.get_label(), final_predictions_xgb)
final_r2_xgb = r2_score(dtest.get_label(), final_predictions_xgb)

# 결과 출력
print(f"XGBoost - Training Time: {training_time_xgb:.2f}s, RMSE: {final_rmse_xgb:.4f}, MSE: {final_mse_xgb:.4f}, MAE: {final_mae_xgb:.4f}, R2: {final_r2_xgb:.4f}")

Best parameters for XGBoost: {'learning_rate': 0.01, 'max_depth': 8, 'subsample': 0.7, 'colsample_bytree': 0.8500000000000001, 'booster': 'gbtree', 'n_estimators': 1000}
XGBoost - Training Time: 4.62s, RMSE: 0.4005, MSE: 0.1604, MAE: 0.2904, R2: 0.8442


In [15]:
# LightGBM
best_params_lgb = study_lgb.best_params
print("Best parameters for LightGBM:", best_params_lgb)

start_time = time.time()
final_model_lgb = lgb.train(best_params_lgb, ltrain_val, num_boost_round=best_params_lgb['n_estimators'])
training_time_lgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_lgb, "./model_/daejeon_lgb.pkl")

# 예측 (test 데이터로 예측)
final_predictions_lgb = final_model_lgb.predict(X_test_rf_gbm)  # X_test_rf_gbm 사용

# 평가 지표 계산
final_rmse_lgb = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_lgb))
final_mse_lgb = mean_squared_error(y_test_rf_gbm, final_predictions_lgb)
final_mae_lgb = mean_absolute_error(y_test_rf_gbm, final_predictions_lgb)
final_r2_lgb = r2_score(y_test_rf_gbm, final_predictions_lgb)

# 결과 출력
print(f"LightGBM - Training Time: {training_time_lgb:.2f}s, RMSE: {final_rmse_lgb:.4f}, MSE: {final_mse_lgb:.4f}, MAE: {final_mae_lgb:.4f}, R2: {final_r2_lgb:.4f}")

Best parameters for LightGBM: {'learning_rate': 0.01, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1000}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000892 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1677
[LightGBM] [Info] Number of data points in the train set: 32142, number of used features: 21
[LightGBM] [Info] Start training from score 1.356206


C:\Users\user1\AppData\Roaming\Python\Python39\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


LightGBM - Training Time: 2.45s, RMSE: 0.4051, MSE: 0.1641, MAE: 0.2927, R2: 0.8406


In [16]:
# CatBoost
best_params_cat = study_cat.best_params
print("Best parameters for CatBoost:", best_params_cat)

start_time = time.time()
final_model_cat = cb.CatBoostRegressor(**best_params_cat, verbose=0, random_seed=SEED)
final_model_cat.fit(ctrain_val)
training_time_cat = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_cat, "./model_/daejeon_cat.pkl")

final_predictions_cat = final_model_cat.predict(ctest)
final_rmse_cat = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_cat))
final_mse_cat = mean_squared_error(y_test_rf_gbm, final_predictions_cat)
final_mae_cat = mean_absolute_error(y_test_rf_gbm, final_predictions_cat)
final_r2_cat = r2_score(y_test_rf_gbm, final_predictions_cat)
print(f"CatBoost - Training Time: {training_time_cat:.2f}s, RMSE: {final_rmse_cat:.4f}, MSE: {final_mse_cat:.4f}, MAE: {final_mae_cat:.4f}, R2: {final_r2_cat:.4f}")

Best parameters for CatBoost: {'learning_rate': 0.04, 'depth': 9, 'l2_leaf_reg': 5}
CatBoost - Training Time: 9.05s, RMSE: 0.4022, MSE: 0.1618, MAE: 0.2905, R2: 0.8429


인천

In [12]:
# Random Forest
best_params_rf = study_rf.best_params
print("Best parameters for Random Forest:", best_params_rf)

# 모델 학습 및 저장
start_time = time.time()
final_model_rf = RandomForestRegressor(**best_params_rf, random_state=SEED)
final_model_rf.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_rf = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_rf, "./model_/incheon_rf.pkl")

# 모델 성능 평가
final_predictions_rf = final_model_rf.predict(X_test_rf_gbm)
final_rmse_rf = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_rf))
final_mse_rf = mean_squared_error(y_test_rf_gbm, final_predictions_rf)
final_mae_rf = mean_absolute_error(y_test_rf_gbm, final_predictions_rf)
final_r2_rf = r2_score(y_test_rf_gbm, final_predictions_rf)

print(f"Random Forest - Training Time: {training_time_rf:.2f}s, RMSE: {final_rmse_rf:.4f}, MSE: {final_mse_rf:.4f}, MAE: {final_mae_rf:.4f}, R2: {final_r2_rf:.4f}")

Best parameters for Random Forest: {'n_estimators': 128, 'max_features': 'log2'}
Random Forest - Training Time: 7.83s, RMSE: 0.4243, MSE: 0.1801, MAE: 0.3198, R2: 0.7893


In [13]:
# Gradient Boosting
best_params_gbm = study_gbm.best_params
print("Best parameters for Gradient Boosting:", best_params_gbm)

start_time = time.time()
final_model_gbm = GradientBoostingRegressor(**best_params_gbm)
final_model_gbm.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_gbm = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_gbm, "./model_/incheon_gbm.pkl")

final_predictions_gbm = final_model_gbm.predict(X_test_rf_gbm)
final_rmse_gbm = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_gbm))
final_mse_gbm = mean_squared_error(y_test_rf_gbm, final_predictions_gbm)
final_mae_gbm = mean_absolute_error(y_test_rf_gbm, final_predictions_gbm)
final_r2_gbm = r2_score(y_test_rf_gbm, final_predictions_gbm)
print(f"Gradient Boosting - Training Time: {training_time_gbm:.2f}s, RMSE: {final_rmse_gbm:.4f}, MSE: {final_mse_gbm:.4f}, MAE: {final_mae_gbm:.4f}, R2: {final_r2_gbm:.4f}")

Best parameters for Gradient Boosting: {'n_estimators': 500, 'learning_rate': 0.03, 'max_depth': 5, 'loss': 'huber'}
Gradient Boosting - Training Time: 40.40s, RMSE: 0.4102, MSE: 0.1682, MAE: 0.2999, R2: 0.8031


In [14]:
# XGBoost
best_params_xgb = study_xgb.best_params
print("Best parameters for XGBoost:", best_params_xgb)

best_num_boost_round = best_params_xgb.pop('n_estimators')
start_time = time.time()
final_model_xgb = xgb.train(best_params_xgb, dtrain_val, num_boost_round=best_num_boost_round)
training_time_xgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_xgb, "./model_/incheon_xgb.pkl")

# 예측 (test 데이터로 예측)
final_predictions_xgb = final_model_xgb.predict(dtest)

# 평가 지표 계산
final_rmse_xgb = np.sqrt(mean_squared_error(dtest.get_label(), final_predictions_xgb))
final_mse_xgb = mean_squared_error(dtest.get_label(), final_predictions_xgb)
final_mae_xgb = mean_absolute_error(dtest.get_label(), final_predictions_xgb)
final_r2_xgb = r2_score(dtest.get_label(), final_predictions_xgb)

# 결과 출력
print(f"XGBoost - Training Time: {training_time_xgb:.2f}s, RMSE: {final_rmse_xgb:.4f}, MSE: {final_mse_xgb:.4f}, MAE: {final_mae_xgb:.4f}, R2: {final_r2_xgb:.4f}")

Best parameters for XGBoost: {'learning_rate': 0.02, 'max_depth': 6, 'subsample': 0.5, 'colsample_bytree': 0.8500000000000001, 'booster': 'gbtree', 'n_estimators': 500}
XGBoost - Training Time: 2.71s, RMSE: 0.4043, MSE: 0.1634, MAE: 0.2994, R2: 0.8088


In [15]:
# LightGBM
best_params_lgb = study_lgb.best_params
print("Best parameters for LightGBM:", best_params_lgb)

start_time = time.time()
final_model_lgb = lgb.train(best_params_lgb, ltrain_val, num_boost_round=best_params_lgb['n_estimators'])
training_time_lgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_lgb, "./model_/incheon_lgb.pkl")

# 예측 (test 데이터로 예측)
final_predictions_lgb = final_model_lgb.predict(X_test_rf_gbm)  # X_test_rf_gbm 사용

# 평가 지표 계산
final_rmse_lgb = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_lgb))
final_mse_lgb = mean_squared_error(y_test_rf_gbm, final_predictions_lgb)
final_mae_lgb = mean_absolute_error(y_test_rf_gbm, final_predictions_lgb)
final_r2_lgb = r2_score(y_test_rf_gbm, final_predictions_lgb)

# 결과 출력
print(f"LightGBM - Training Time: {training_time_lgb:.2f}s, RMSE: {final_rmse_lgb:.4f}, MSE: {final_mse_lgb:.4f}, MAE: {final_mae_lgb:.4f}, R2: {final_r2_lgb:.4f}")

Best parameters for LightGBM: {'learning_rate': 0.03, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'dart', 'n_estimators': 1000}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001736 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1803
[LightGBM] [Info] Number of data points in the train set: 32142, number of used features: 21
[LightGBM] [Info] Start training from score 1.143254


C:\Users\user1\AppData\Roaming\Python\Python39\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


LightGBM - Training Time: 15.21s, RMSE: 0.4133, MSE: 0.1708, MAE: 0.3121, R2: 0.8001


In [16]:
# CatBoost
best_params_cat = study_cat.best_params
print("Best parameters for CatBoost:", best_params_cat)

start_time = time.time()
final_model_cat = cb.CatBoostRegressor(**best_params_cat, verbose=0, random_seed=SEED)
final_model_cat.fit(ctrain_val)
training_time_cat = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_cat, "./model_/incheon_cat.pkl")

final_predictions_cat = final_model_cat.predict(ctest)
final_rmse_cat = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_cat))
final_mse_cat = mean_squared_error(y_test_rf_gbm, final_predictions_cat)
final_mae_cat = mean_absolute_error(y_test_rf_gbm, final_predictions_cat)
final_r2_cat = r2_score(y_test_rf_gbm, final_predictions_cat)
print(f"CatBoost - Training Time: {training_time_cat:.2f}s, RMSE: {final_rmse_cat:.4f}, MSE: {final_mse_cat:.4f}, MAE: {final_mae_cat:.4f}, R2: {final_r2_cat:.4f}")

Best parameters for CatBoost: {'learning_rate': 0.03, 'depth': 9, 'l2_leaf_reg': 8}
CatBoost - Training Time: 9.21s, RMSE: 0.3956, MSE: 0.1565, MAE: 0.2919, R2: 0.8169


서울

In [12]:
# Random Forest
best_params_rf = study_rf.best_params
print("Best parameters for Random Forest:", best_params_rf)

# 모델 학습 및 저장
start_time = time.time()
final_model_rf = RandomForestRegressor(**best_params_rf, random_state=SEED)
final_model_rf.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_rf = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_rf, "./model_/seoul_rf.pkl")

# 모델 성능 평가
final_predictions_rf = final_model_rf.predict(X_test_rf_gbm)
final_rmse_rf = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_rf))
final_mse_rf = mean_squared_error(y_test_rf_gbm, final_predictions_rf)
final_mae_rf = mean_absolute_error(y_test_rf_gbm, final_predictions_rf)
final_r2_rf = r2_score(y_test_rf_gbm, final_predictions_rf)

print(f"Random Forest - Training Time: {training_time_rf:.2f}s, RMSE: {final_rmse_rf:.4f}, MSE: {final_mse_rf:.4f}, MAE: {final_mae_rf:.4f}, R2: {final_r2_rf:.4f}")

Best parameters for Random Forest: {'n_estimators': 128, 'max_features': 'log2'}
Random Forest - Training Time: 7.70s, RMSE: 0.4369, MSE: 0.1909, MAE: 0.3247, R2: 0.7745


In [13]:
# Gradient Boosting
best_params_gbm = study_gbm.best_params
print("Best parameters for Gradient Boosting:", best_params_gbm)

start_time = time.time()
final_model_gbm = GradientBoostingRegressor(**best_params_gbm)
final_model_gbm.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_gbm = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_gbm, "./model_/seoul_gbm.pkl")

final_predictions_gbm = final_model_gbm.predict(X_test_rf_gbm)
final_rmse_gbm = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_gbm))
final_mse_gbm = mean_squared_error(y_test_rf_gbm, final_predictions_gbm)
final_mae_gbm = mean_absolute_error(y_test_rf_gbm, final_predictions_gbm)
final_r2_gbm = r2_score(y_test_rf_gbm, final_predictions_gbm)
print(f"Gradient Boosting - Training Time: {training_time_gbm:.2f}s, RMSE: {final_rmse_gbm:.4f}, MSE: {final_mse_gbm:.4f}, MAE: {final_mae_gbm:.4f}, R2: {final_r2_gbm:.4f}")

Best parameters for Gradient Boosting: {'n_estimators': 500, 'learning_rate': 0.03, 'max_depth': 6, 'loss': 'huber'}
Gradient Boosting - Training Time: 50.83s, RMSE: 0.4245, MSE: 0.1802, MAE: 0.3075, R2: 0.7870


In [14]:
# XGBoost
best_params_xgb = study_xgb.best_params
print("Best parameters for XGBoost:", best_params_xgb)

best_num_boost_round = best_params_xgb.pop('n_estimators')
start_time = time.time()
final_model_xgb = xgb.train(best_params_xgb, dtrain_val, num_boost_round=best_num_boost_round)
training_time_xgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_xgb, "./model_/seoul_xgb.pkl")

# 예측 (test 데이터로 예측)
final_predictions_xgb = final_model_xgb.predict(dtest)

# 평가 지표 계산
final_rmse_xgb = np.sqrt(mean_squared_error(dtest.get_label(), final_predictions_xgb))
final_mse_xgb = mean_squared_error(dtest.get_label(), final_predictions_xgb)
final_mae_xgb = mean_absolute_error(dtest.get_label(), final_predictions_xgb)
final_r2_xgb = r2_score(dtest.get_label(), final_predictions_xgb)

# 결과 출력
print(f"XGBoost - Training Time: {training_time_xgb:.2f}s, RMSE: {final_rmse_xgb:.4f}, MSE: {final_mse_xgb:.4f}, MAE: {final_mae_xgb:.4f}, R2: {final_r2_xgb:.4f}")

Best parameters for XGBoost: {'learning_rate': 0.04, 'max_depth': 7, 'subsample': 0.6, 'colsample_bytree': 1.0, 'booster': 'dart', 'n_estimators': 250}
XGBoost - Training Time: 19.26s, RMSE: 0.4222, MSE: 0.1783, MAE: 0.3077, R2: 0.7893


In [15]:
# LightGBM
best_params_lgb = study_lgb.best_params
print("Best parameters for LightGBM:", best_params_lgb)

start_time = time.time()
final_model_lgb = lgb.train(best_params_lgb, ltrain_val, num_boost_round=best_params_lgb['n_estimators'])
training_time_lgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_lgb, "./model_/seoul_lgb.pkl")

# 예측 (test 데이터로 예측)
final_predictions_lgb = final_model_lgb.predict(X_test_rf_gbm)  # X_test_rf_gbm 사용

# 평가 지표 계산
final_rmse_lgb = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_lgb))
final_mse_lgb = mean_squared_error(y_test_rf_gbm, final_predictions_lgb)
final_mae_lgb = mean_absolute_error(y_test_rf_gbm, final_predictions_lgb)
final_r2_lgb = r2_score(y_test_rf_gbm, final_predictions_lgb)

# 결과 출력
print(f"LightGBM - Training Time: {training_time_lgb:.2f}s, RMSE: {final_rmse_lgb:.4f}, MSE: {final_mse_lgb:.4f}, MAE: {final_mae_lgb:.4f}, R2: {final_r2_lgb:.4f}")

Best parameters for LightGBM: {'learning_rate': 0.01, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1000}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001211 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1897
[LightGBM] [Info] Number of data points in the train set: 32142, number of used features: 21
[LightGBM] [Info] Start training from score 1.115901


C:\Users\user1\AppData\Roaming\Python\Python39\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


LightGBM - Training Time: 3.04s, RMSE: 0.4218, MSE: 0.1779, MAE: 0.3062, R2: 0.7897


In [16]:
# CatBoost
best_params_cat = study_cat.best_params
print("Best parameters for CatBoost:", best_params_cat)

start_time = time.time()
final_model_cat = cb.CatBoostRegressor(**best_params_cat, verbose=0, random_seed=SEED)
final_model_cat.fit(ctrain_val)
training_time_cat = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_cat, "./model_/seoul_cat.pkl")

final_predictions_cat = final_model_cat.predict(ctest)
final_rmse_cat = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_cat))
final_mse_cat = mean_squared_error(y_test_rf_gbm, final_predictions_cat)
final_mae_cat = mean_absolute_error(y_test_rf_gbm, final_predictions_cat)
final_r2_cat = r2_score(y_test_rf_gbm, final_predictions_cat)
print(f"CatBoost - Training Time: {training_time_cat:.2f}s, RMSE: {final_rmse_cat:.4f}, MSE: {final_mse_cat:.4f}, MAE: {final_mae_cat:.4f}, R2: {final_r2_cat:.4f}")

Best parameters for CatBoost: {'learning_rate': 0.03, 'depth': 9, 'l2_leaf_reg': 2}
CatBoost - Training Time: 10.00s, RMSE: 0.4180, MSE: 0.1747, MAE: 0.3058, R2: 0.7935
